# ทดสอบ Face API - คู่มือสำหรับมือใหม่

**สำหรับ Person 3: ผู้ดูแล Face Expression Detection**

## เกี่ยวกับ Notebook นี้

Notebook นี้ใช้สำหรับทดสอบ Face API ก่อนนำไปใช้ใน Frontend:
- ตรวจจับอารมณ์ (Happy, Sad, Angry, Surprised, Neutral, etc.)
- ตรวจจับใบหน้า (Face Detection)
- วัดความแม่นยำ (Confidence Score)

## วัตถุประสงค์

1. **ทดสอบโมเดลก่อนใช้งานจริง** - ตรวจสอบว่าโมเดลทำงานได้ดี
2. **วัดความแม่นยำ** - ดู confidence score ต้องสูงกว่า 50%
3. **ปรับปรุงพารามิเตอร์** - หาค่าที่เหมาะสมสำหรับโปรเจค

## ขั้นตอนการทำงาน

1. ตรวจสอบไฟล์โมเดล Face API
2. โหลดและทดสอบโมเดล  
3. ทดสอบกับรูปภาพตัวอย่าง
4. ปรับปรุงความแม่นยำ
5. เชื่อมต่อกับระบบหลัก

In [ ]:
# 📦 ขั้นตอนที่ 1: ติดตั้ง Libraries ที่จำเป็น
print("🔄 กำลังติดตั้ง libraries สำหรับ Face Detection...")
print("⏰ กรุณารอสักครู่ (อาจใช้เวลา 2-3 นาที)")

import sys
!{sys.executable} -m pip install opencv-python pillow numpy matplotlib requests
!{sys.executable} -m pip install mediapipe  # สำหรับ Face Detection

print("\n" + "="*60)
print("✅ ติดตั้งเสร็จแล้ว! พร้อมใช้งาน")
print("📋 Libraries ที่ติดตั้ง:")
print("   - OpenCV: สำหรับประมวลผลภาพ") 
print("   - Pillow: สำหรับจัดการรูปภาพ")
print("   - NumPy: สำหรับคำนวณ")
print("   - Matplotlib: สำหรับแสดงกราฟ")
print("   - MediaPipe: สำหรับ Face Detection")
print("="*60)

In [ ]:
# 📚 ขั้นตอนที่ 2: Import Libraries และสร้าง Face Detector

print("📖 กำลัง import libraries...")
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import mediapipe as mp

print("="*60)
print("🖥️  ข้อมูลระบบ:")
print(f"   📁 Current Directory: {os.getcwd()}")
print(f"   📸 OpenCV Version: {cv2.__version__}")
print(f"   🤖 MediaPipe Version: {mp.__version__}")

class FaceEmotionDetector:
    """
    คลาสสำหรับตรวจจับใบหน้าและอารมณ์
    ใช้ MediaPipe สำหรับการตรวจจับใบหน้าเบื้องต้น
    """
    
    def __init__(self):
        print("🔧 สร้าง Face Emotion Detector...")
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0,  # 0 สำหรับภาพใกล้ (< 2 เมตร)
            min_detection_confidence=0.5
        )
        print("✅ Face Detector พร้อมแล้ว!")
        
    def detect_faces(self, image_path_or_array, show_details=True):
        """ตรวจจับใบหน้าจากรูปภาพ"""
        try:
            # อ่านรูปภาพ
            if isinstance(image_path_or_array, str):
                image = cv2.imread(image_path_or_array)
                if image is None:
                    print(f"❌ ไม่สามารถอ่านไฟล์ {image_path_or_array}")
                    return None
            else:
                image = image_path_or_array.copy()
            
            # แปลงเป็น RGB (MediaPipe ต้องการ RGB)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # ตรวจจับใบหน้า
            results = self.face_detection.process(rgb_image)
            
            detected_faces = []
            
            if results.detections:
                for i, detection in enumerate(results.detections):
                    confidence = detection.score[0]
                    
                    # ได้ตำแหน่งใบหน้า
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, ic = image.shape
                    x = int(bboxC.xmin * iw)
                    y = int(bboxC.ymin * ih)
                    w = int(bboxC.width * iw)
                    h = int(bboxC.height * ih)
                    
                    detected_faces.append({
                        'face_id': i + 1,
                        'confidence': confidence,
                        'bbox': (x, y, w, h),
                        'emotion': 'neutral'  # ค่าเริ่มต้น (ต้องใช้โมเดลเพิ่มเติมสำหรับอารมณ์)
                    })
                    
                    # วาดกรอบรอบใบหน้า
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, f'Face {i+1}: {confidence:.2f}', 
                               (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            result = {
                'total_faces': len(detected_faces),
                'faces': detected_faces,
                'image_with_boxes': image
            }
            
            if show_details:
                print("="*50)
                print("👤 ผลการตรวจจับใบหน้า:")
                print(f"   📊 จำนวนใบหน้าที่พบ: {len(detected_faces)}")
                
                for face in detected_faces:
                    print(f"   👤 ใบหน้าที่ {face['face_id']}: ความแม่นยำ {face['confidence']:.1%}")
                
                if len(detected_faces) == 0:
                    print("   ⚠️  ไม่พบใบหน้าในรูปภาพ")
                    print("   💡 คำแนะนำ: ถ่ายรูปให้ใบหน้าชัดเจน หันหน้าเข้ากล้อง")
                
                print("="*50)
            
            return result
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            return None
    
    def get_emotion_analysis(self, face_region):
        """
        วิเคราะห์อารมณ์จากใบหน้า (Mock function)
        ในการใช้งานจริง ต้องใช้โมเดลเพิ่มเติม
        """
        # Mock emotions - ในการใช้งานจริงต้องใช้โมเดล AI
        emotions = {
            'neutral': 0.4,
            'happy': 0.3,
            'sad': 0.1,
            'angry': 0.1,
            'surprised': 0.1
        }
        
        # หาอารมณ์ที่มี score สูงสุด
        top_emotion = max(emotions, key=emotions.get)
        
        return {
            'emotions': emotions,
            'top_emotion': top_emotion,
            'confidence': emotions[top_emotion]
        }

# สร้าง instance
print("🚀 กำลังสร้าง Face Detector...")
face_detector = FaceEmotionDetector()

print("\n🎉 พร้อมทดสอบ Face Detection!")
print("📋 ฟังก์ชันที่พร้อมใช้:")
print("   - face_detector.detect_faces() - ตรวจจับใบหน้า")
print("   - face_detector.get_emotion_analysis() - วิเคราะห์อารมณ์")

In [ ]:
# 🖼️ ขั้นตอนที่ 3: ทดสอบด้วยรูปภาพ

def test_face_detection_with_image(image_filename="test_face.jpg"):
    """
    ทดสอบการตรวจจับใบหน้าด้วยไฟล์รูปภาพ
    
    วิธีใช้:
    1. วางไฟล์รูปภาพในโฟลเดอร์เดียวกับ notebook นี้
    2. เปลี่ยนชื่อไฟล์ใน function นี้
    3. รัน cell
    """
    
    print("🔍 กำลังค้นหาไฟล์รูปภาพ...")
    print(f"📄 ชื่อไฟล์: {image_filename}")
    
    if os.path.exists(image_filename):
        print(f"✅ พบไฟล์ {image_filename}")
        
        # อ่านและแสดงรูปภาพต้นฉบับ
        original_image = cv2.imread(image_filename)
        original_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        
        # ทดสอบการตรวจจับใบหน้า
        print("\n🤖 กำลังตรวจจับใบหน้า...")
        result = face_detector.detect_faces(image_filename, show_details=True)
        
        if result:
            # แสดงผลเปรียบเทียบ
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # รูปต้นฉบับ
            ax1.imshow(original_rgb)
            ax1.set_title('📸 รูปภาพต้นฉบับ', fontsize=14)
            ax1.axis('off')
            
            # รูปที่ตรวจจับใบหน้าแล้ว
            result_rgb = cv2.cvtColor(result['image_with_boxes'], cv2.COLOR_BGR2RGB)
            ax2.imshow(result_rgb)
            ax2.set_title(f'👤 ตรวจจับได้ {result["total_faces"]} ใบหน้า', fontsize=14)
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # สรุปผลการทดสอบ
            print("\n🎉 สรุปผลการทดสอบ Face Detection:")
            if result['total_faces'] > 0:
                print(f"   🌟 สำเร็จ! ตรวจจับใบหน้าได้ {result['total_faces']} ใบหน้า")
                
                # วิเคราะห์อารมณ์ (Mock)
                for i, face in enumerate(result['faces']):
                    emotion_result = face_detector.get_emotion_analysis(None)  # Mock
                    print(f"   😊 ใบหน้าที่ {i+1}: อารมณ์ '{emotion_result['top_emotion']}' ({emotion_result['confidence']:.1%})")
                
                print("   💡 โมเดลทำงานได้ดี พร้อมใช้งานใน Frontend")
            else:
                print("   ⚠️  ไม่พบใบหน้าในรูปภาพ")
                print("   💡 คำแนะนำ:")
                print("      - ถ่ายรูปใหม่ให้ใบหน้าชัดเจน")
                print("      - หันหน้าเข้ากล้อง")
                print("      - ทดสอบในแสงที่ดี")
            
            # แสดงกราฟอารมณ์
            if result['total_faces'] > 0:
                emotion_data = face_detector.get_emotion_analysis(None)
                
                plt.figure(figsize=(10, 6))
                emotions = list(emotion_data['emotions'].keys())
                scores = list(emotion_data['emotions'].values())
                
                colors = ['#ff6b6b' if emotion == emotion_data['top_emotion'] else '#4ecdc4' for emotion in emotions]
                
                plt.bar(emotions, scores, color=colors)
                plt.xlabel('Emotions')
                plt.ylabel('Confidence Score')
                plt.title('😊 Face Emotion Analysis (Mock Data)', fontsize=14, pad=20)
                plt.ylim(0, 1)
                
                for i, score in enumerate(scores):
                    plt.text(i, score + 0.01, f'{score:.1%}', ha='center')
                
                plt.tight_layout()
                plt.show()
                
    else:
        print(f"❌ ไม่พบไฟล์ {image_filename}")
        print("\n💡 วิธีเพิ่มรูปทดสอบ:")
        print("   1. ถ่ายรูปใบหน้าที่ต้องการทดสอบ")
        print("   2. บันทึกรูปในโฟลเดอร์เดียวกับ notebook นี้")
        print("   3. เปลี่ยนชื่อไฟล์ในฟังก์ชันนี้")
        print(f"   4. ไฟล์ที่รองรับ: .jpg, .jpeg, .png")

# ทดสอบทันที
print("🚀 เริ่มทดสอบ Face Detection ด้วยรูปภาพ...")
test_face_detection_with_image("test_face.jpg")

print("\n" + "="*60)
print("💡 เคล็ดลับการทดสอบ Face Detection:")
print("   📸 ถ่ายรูปใบหน้าที่ชัดเจน")
print("   💡 ทดสอบในแสงที่ดี")
print("   🔄 ลองหลายอารมณ์เพื่อทดสอบการตรวจจับ")
print("   📊 confidence > 50% = ผ่าน")
print("="*60)

In [ ]:
# 📹 ขั้นตอนที่ 4: ทดสอบด้วยกล้องเว็บแคมแบบ Real-time

def run_face_detection_webcam():
    """
    ทดสอบการตรวจจับใบหน้าด้วยกล้องเว็บแคมแบบ Real-time
    
    คำแนะนำ:
    - กด 'S' เพื่อถ่ายรูปและทดสอบ
    - กด 'Q' เพื่อออกจากระบบ
    - หันหน้าเข้ากล้อง ทำหน้าต่างๆ
    """
    
    print("📹 กำลังเปิดกล้องเว็บแคม สำหรับ Face Detection...")
    
    try:
        # เปิดกล้อง
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("❌ ไม่สามารถเปิดกล้องได้!")
            print("💡 วิธีแก้ไข:")
            print("   - ตรวจสอบว่ากล้องเชื่อมต่ออยู่")
            print("   - ปิดโปรแกรมอื่นที่ใช้กล้อง")
            print("   - รีสตาร์ท Jupyter Notebook")
            return
        
        print("✅ กล้องพร้อมใช้งาน!")
        print("\n" + "="*60)
        print("🎮 การควบคุม:")
        print("   📸 กด 'S' = ถ่ายรูปและทดสอบ Face Detection")
        print("   🚪 กด 'Q' = ออกจากระบบ")
        print("\n😊 ลองทำสีหน้าต่างๆ:")
        print("   😊 ยิ้ม   😢 เศร้า   😠 โกรธ   😲 ประหลาดใจ   😐 เฉยๆ")
        print("="*60)
        
        capture_count = 0
        detection_log = []  # เก็บบันทึกการทดสอบ
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ ไม่สามารถอ่านภาพจากกล้องได้")
                break
            
            # แสดงข้อความบนหน้าจอ
            cv2.putText(frame, "Face Detection Test", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, "Press 'S' to test, 'Q' to quit", 
                       (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"Tests: {capture_count}", 
                       (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            # แสดงกรอบสีม่วงเพื่อให้รู้ว่าเป็น Face Detection
            cv2.rectangle(frame, (50, 50), (590, 430), (255, 0, 255), 2)
            cv2.putText(frame, "Face Detection Zone", 
                       (60, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            
            cv2.imshow('Face Detection - Webcam Test', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('s') or key == ord('S'):  # ทดสอบ
                capture_count += 1
                print(f"\n📸 การทดสอบ Face Detection ครั้งที่ {capture_count}")
                
                # ทดสอบการตรวจจับใบหน้า
                print("🤖 กำลังตรวจจับใบหน้า...")
                result = face_detector.detect_faces(frame, show_details=False)
                
                if result:
                    faces_detected = result['total_faces']
                    
                    # บันทึกผล
                    detection_log.append({
                        'test': capture_count,
                        'faces_detected': faces_detected,
                        'success': faces_detected > 0
                    })
                    
                    # แสดงผลใน terminal
                    print(f"👤 ผล: ตรวจจับใบหน้าได้ {faces_detected} ใบหน้า")
                    
                    # แสดงผลบนภาพ
                    if faces_detected > 0:
                        result_text = f"Faces: {faces_detected}"
                        cv2.putText(frame, result_text, 
                                   (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                        
                        status_text = "Detection Success!"
                        color = (0, 255, 0)  # เขียว
                        print("   ✅ ตรวจจับใบหน้าสำเร็จ!")
                        
                        # Mock emotion analysis
                        emotion_result = face_detector.get_emotion_analysis(None)
                        emotion_text = f"Emotion: {emotion_result['top_emotion']}"
                        cv2.putText(frame, emotion_text, 
                                   (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
                        print(f"   😊 อารมณ์: {emotion_result['top_emotion']} ({emotion_result['confidence']:.1%})")
                        
                    else:
                        result_text = "No Face Detected"
                        cv2.putText(frame, result_text, 
                                   (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 100, 255), 2)
                        
                        status_text = "Try Again"
                        color = (0, 100, 255)  # ส้ม
                        print("   ⚠️  ไม่พบใบหน้า ลองหันหน้าเข้ากล้อง")
                    
                    cv2.putText(frame, status_text, 
                               (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    
                    # แสดงคำแนะนำ
                    tips = ["Try smiling!", "Make angry face", "Look surprised", "Stay neutral"]
                    tip = tips[capture_count % len(tips)]
                    print(f"   💡 ลองครั้งถัดไป: {tip}")
                    cv2.putText(frame, f"Next: {tip}", 
                               (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    # แสดงผลเป็นเวลา 3 วินาที
                    cv2.imshow('Face Detection - Result', frame)
                    cv2.waitKey(3000)
                
            elif key == ord('q') or key == ord('Q'):  # ออกจากโปรแกรม
                print("\n👋 ปิดกล้อง...")
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        # สรุปผลการทดสอบ
        print("\n" + "="*60)
        print("📊 สรุปผลการทดสอบ Face Detection:")
        print(f"   🔢 จำนวนการทดสอบทั้งหมด: {capture_count} ครั้ง")
        
        if detection_log:
            successful_detections = [log for log in detection_log if log['success']]
            success_rate = len(successful_detections) / len(detection_log) * 100
            
            print(f"   ✅ ตรวจจับสำเร็จ: {len(successful_detections)} ครั้ง")
            print(f"   📈 อัตราความสำเร็จ: {success_rate:.1f}%")
            
            print("\n📝 รายละเอียดการทดสอบ:")
            for log in detection_log:
                status = "✅" if log['success'] else "⚠️"
                print(f"   {status} ครั้งที่ {log['test']:2d}: {log['faces_detected']} ใบหน้า")
            
            if success_rate >= 80:
                print("\n🎉 ยอดเยี่ยม! Face Detection ทำงานได้ดีมาก")
            elif success_rate >= 60:
                print("\n✅ ดี! Face Detection ทำงานได้ดี")
            else:
                print("\n⚠️  ควรปรับปรุง Face Detection")
                print("💡 คำแนะนำ: ทดสอบในแสงที่ดีขึ้น")
        
        print("="*60)
        print("✅ เสร็จสิ้นการทดสอบ Face Detection")
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับกล้อง: {e}")
        print("💡 วิธีแก้ไข:")
        print("   1. รีสตาร์ท Jupyter Notebook")
        print("   2. ตรวจสอบว่ากล้องทำงานปกติ")
        print("   3. ปิดโปรแกรมอื่นที่ใช้กล้อง")

print("📹 ฟังก์ชันทดสอบกล้อง Face Detection พร้อมใช้งาน")
print("\n💡 วิธีเริ่มทดสอบ:")
print("   รันคำสั่ง: run_face_detection_webcam()")

print("\n🔥 คำแนะนำสำหรับการทดสอบ Face Detection:")
print("   1. หันหน้าเข้ากล้องให้ชัดเจน")
print("   2. ทดสอบในแสงที่ดี")  
print("   3. ลองทำสีหน้าต่างๆ เพื่อทดสอบ")
print("   4. อยู่ในกรอบสีม่วงเพื่อผลที่ดีที่สุด")
print("   5. บันทึกผลเพื่อประเมินการใช้งานจริง")

print("\n" + "="*60)
print("🚀 พิมพ์คำสั่งนี้เพื่อเริ่มทดสอบ Face Detection:")
print("run_face_detection_webcam()")
print("="*60)

## 🎯 สรุปและขั้นตอนถัดไป

### ✅ สิ่งที่คุณควรได้จากการทดสอบ:

1. **ตรวจจับใบหน้าได้ ≥ 80%** - ระบบพร้อมใช้งาน
2. **ตรวจจับได้ใน Real-time** - ไม่ช้า ไม่กระตุก
3. **วิเคราะห์อารมณ์ได้** - อย่างน้อยแยกได้ 3-5 อารมณ์

### 🔄 หากผลการทดสอบไม่ดี:

**ปัญหา: ตรวจจับใบหน้าไม่ได้**
- 💡 ปรับค่า `min_detection_confidence` ให้ต่ำลง (0.3-0.4)
- 📸 ทดสอบในแสงที่ดีขึ้น
- 🎥 ตรวจสอบคุณภาพกล้อง

**ปัญหา: ช้าเกินไป**
- 🔧 ใช้ `model_selection=0` (สำหรับภาพใกล้)
- 📏 ลดขนาดภาพก่อนประมวลผล
- ⚡ ใช้ GPU หากมี

**ปัญหา: อารมณ์ไม่แม่นยำ**
- 🤖 ต้องใช้โมเดล AI เพิ่มเติมสำหรับ Emotion Recognition
- 📊 ในตัวอย่างนี้ใช้ Mock data เท่านั้น

---

### 🚀 ขั้นตอนนำไปใช้ใน Frontend:

1. **ติดตั้ง face-api.js** ใน Frontend project
2. **ดาวน์โหลดโมเดล** วางในโฟลเดอร์ `public/face-models/`
3. **เขียน JavaScript Code** ตามคู่มือใน README.md
4. **ทดสอบใน Browser** ร่วมกับ Hand Detection

---

### 📞 ติดต่อทีมเมื่อ:

- **Person 1 & 2:** ประสานการทำงานร่วมกับ Hand Models
- **Person 4:** ส่งข้อมูลอารมณ์ไป Backend/LLM
- **ทีม:** รายงานผลการทดสอบและปัญหาที่พบ

---

### 🎉 เมื่อระบบพร้อมแล้ว:

✅ **Face Detection ทำงานได้ดี**  
✅ **Emotion Analysis พื้นฐานพร้อม**  
✅ **ทดสอบใน Real-time สำเร็จ**  
✅ **พร้อมผสานเข้า Frontend**  

**🎊 ยินดีด้วย! Face Detection System ของคุณพร้อมใช้งานแล้ว**

---

### 💡 หมายเหตุสำคัญ:

- **Emotion Recognition** ในตัวอย่างนี้ใช้ Mock data
- **สำหรับการใช้งานจริง** แนะนำให้ใช้ face-api.js หรือ Azure Face API
- **MediaPipe** ใช้สำหรับ Face Detection เบื้องต้นเท่านั้น
- **ประสิทธิภาพ** จะดีขึ้นหากใช้โมเดลที่เหมาะสมกับ Browser